<a href="https://colab.research.google.com/github/pablitomarie/LSRL_Bibliography-Project/blob/main/LSRL_notebook_CZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [3]:
import string

In [4]:
import pandas as pd
import nltk
import numpy as np

In [5]:
from collections import Counter

In [6]:
from nltk.corpus import stopwords
nltk.download('stopwords')

stopword = set(stopwords.words('english'))

from string import punctuation

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

# Open up sheet

In [8]:
# open LSRL_Master2
sheet = gc.open('LSRL_Master2')
wsRaw = sheet.get_worksheet(0)

# turn sheet into Pandas dataframe
df = pd.DataFrame(wsRaw.get_all_records())

## Useful Functions 

## Clean Up Sheet

In [9]:
df.shape

(1063, 87)

In [10]:
df['References'] #empty is "" not NaN

0        
1        
2        
3        
4        
       ..
1058     
1059     
1060     
1061     
1062     
Name: References, Length: 1063, dtype: object

In [11]:
# replace empty with NaN
df = df.replace('', np.nan)

### 1. Removing null columns



In [12]:
# drop all columns (axis = 1) with all null values
df = df.dropna(axis = 1, how = 'all')

In [13]:
# new sheet has 44 columns
df.shape

(1063, 44)

In [14]:
# remove rows with no publications
df['key'].isnull().sum()
df.dropna(subset = ['key'], inplace = True) # remove them for now
df.loc[len(df)-1] # get the last item should be Ernesto R.

<ipython-input-14-4898bd0a70b1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset = ['key'], inplace = True) # remove them for now


                                                                      NaN
Item Type                                                 conferencePaper
Publication Year                                                     2023
Author                                       Gutiérrez Topete, Ernesto R.
Author 1                                     Gutiérrez Topete, Ernesto R.
Author 2                                                              NaN
Author 3                                                              NaN
Author 4                                                              NaN
Author 5                                                              NaN
Author 6                                                              NaN
Title                        Frequency and efficiency in Spanish proverbs
key                          Frequency and efficiency in Spanish proverbs
Abstract                Zipf’s law states that there is an inverse rel...
doi url                            htt

In [15]:
# new sheet has 1063 rows
df.shape

(1063, 44)

### 2. Finding and removing duplicates

In [16]:
# Finding duplicates
df[df.duplicated(subset = ['key'])]

,,Item Type,Publication Year,Author,Author 1,Author 2,Author 3,Author 4,Author 5,Author 6,...,Library Catalog,Extra,Notes,Publication Type 2,File Attachments,Manual Tags,Editor,Meeting Name,Meeting Location,Conference Name
113,FKDHS8XD,bookSection,1978,"Reynolds, L.; Mez-Dombkowski, E.","Reynolds, L.","Mez-Dombkowski, E.",NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,book article,NaN,NaN,"Morin, YC",LSRL06,Université de Québec à Montréal,Linguistic Symposium on Romance Languages
116,LVXHD7LB,bookSection,1978,"Saltarelli, M.","Saltarelli, M.",NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,book article,NaN,NaN,"Morin, YC",LSRL06,Université de Québec à Montréal,Linguistic Symposium on Romance Languages
117,WDK32N3G,bookSection,1978,"Walker, D.","Walker, D.",NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,book article,NaN,NaN,"Morin, YC",LSRL06,Université de Québec à Montréal,Linguistic Symposium on Romance Languages
118,2GV9EJBE,bookSection,1978,"Stead, D.","Stead, D.",NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,book article,NaN,NaN,"Morin, YC",LSRL06,Université de Québec à Montréal,Linguistic Symposium on Romance Languages
121,YHESVZTV,bookSection,1978,"Ford, A.","Ford, A.",NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,book article,NaN,NaN,"Morin, YC",LSRL06,Université de Québec à Montréal,Linguistic Symposium on Romance Languages
126,6IFXAMKH,bookSection,1978,"Luján, M.","Luján, M.",NaN,NaN,NaN,NaN,NaN,...,NaN,102.0,NaN,book article,NaN,NaN,"Morin, YC",LSRL06,Université de Québec à Montréal,Linguistic Symposium on Romance Languages


In [17]:
# Remove duplicates
df = df.drop_duplicates(subset = ['key'])
df.shape

(1057, 44)

### 3. Fix page numbers

In [18]:
# Fix date issue 

# abbreviated months
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# fill null values with -1 
df1 = df[['Begin Page', 'End Page']].fillna(value = '-1')

# if endpage has month then swap and put into new columns: 11-jan to jan-11 - so now only beginpage has month
df1['beginpage'] = np.where(df1['End Page'].isin(months), df1['End Page'], df1['Begin Page'])
df1['endpage'] = np.where(df1['End Page'].isin(months), df1['Begin Page'], df1['End Page'])

# if see month in startpage replace with numbers
for i,val in enumerate(months):
     df1['beginpage'] = df1['beginpage'].replace(val, str(i+1))

In [19]:
# Add truncated 1st number to end pages
df1['endpage'] = np.where(df1['beginpage'].astype('int') > df1['endpage'].astype('int'), 
                          df1['beginpage'].astype('string').str[:1] + df1['endpage'].astype('string') , df1['endpage'].astype('string'))

In [20]:
# replace Pages column
df1['Pages'] = df1['beginpage'].astype('string') + "-" + df1['endpage'].astype('string')

In [21]:
# fill in num pages column
df1['Num Pages'] = df1['endpage'].astype('int') - df1['beginpage'].astype('int')

In [22]:
# replace and add to new data frame
df2 = df.copy()
df2['Pages'] = df1['Pages']
df2['Begin Page'] = df1['beginpage']
df2['End Page'] = df1['endpage']
df2.insert(loc = (df2.columns.get_loc('End Page')+1), column = 'Num Pages', value = df1['Num Pages'])

In [23]:
# replace null values with ""
df2 = df2.replace('-1', '')
df2 = df2.replace('-1--1', '')
df2 = df2.fillna(value = '')

In [24]:
# grab values from df
values = [df2.columns.values.tolist()] + df2.values.tolist()

# update worksheet
#wsResults = sheet.add_worksheet(title="Results", rows=1500, cols=50)
wsResults = sheet.get_worksheet(1)
wsResults.update(None, values)

{'spreadsheetId': '1g81DRB2T954vR6ibqA8ngYdlD5nuoRlsifLKPRLAV6k',
 'updatedRange': 'Cleaned!A1:AS1058',
 'updatedRows': 1058,
 'updatedColumns': 45,
 'updatedCells': 47610}

## Fun Part! 

### Titles
- avg length of titles
- longest title
- shortest title
- search articles that include a certain word
- counts of unique titles
- most common words used in titles